In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, "../src/")

In [1]:
import random
import numpy as np
import skimpy
import skimpy_3d
from skimpy_blox import minecraft
from skimpy import functional as F

In [2]:
level = minecraft.SkimpyMinecraftLevel.load("../data/vanillaburp.pickle.gz")

In [3]:
t = level.megatensor()

In [4]:
for i in range(20):
    print(f"{i}=>{F.sum((t == i).to(int))}")

0=>22615704
1=>12336
2=>6168
3=>74016
4=>0
5=>0
6=>0
7=>0
8=>0
9=>0
10=>0
11=>0
12=>0
13=>0
14=>0
15=>0
16=>0
17=>0
18=>0
19=>0


In [5]:
config = skimpy_3d.VoxelConfig()
config[0] = skimpy_3d.EmptyVoxel()
for i in range(1, 13):
    config[i] = skimpy_3d.ColorVoxel(
        random.randint(0, 255),
        random.randint(0, 255),
        random.randint(0, 255),
    )

In [6]:
%%time
mesh = skimpy_3d.generate_mesh(config, t._tensor)

Wall time: 46.5 ms


In [7]:
len(mesh.triangles)

112536

In [8]:
from pythreejs import *
import numpy as np
from IPython.display import display

In [9]:
geometry = BufferGeometry(
    attributes={
        "position": BufferAttribute(
            np.array(mesh.positions, dtype='float32').reshape(-1, 3),
            normalized=False,
        ),
        "index": BufferAttribute(
            np.array(mesh.triangles, dtype='uint32'),
            normalized=False,
        ),
        "color": BufferAttribute(
            np.array(mesh.colors, dtype='float32').reshape(-1, 3) / 255.0,
        ),
    },
)

In [10]:
geometry.exec_three_obj_method("computeVertexNormals")

In [11]:
js_mesh = Mesh(
    geometry=geometry,
    material=MeshLambertMaterial(vertexColors="VertexColors"),
    position=[0, 0, 0],
)

In [12]:
camera = PerspectiveCamera(position=[300, 50, 1200], fov=20)

point_light = PointLight(color='#ffffff', position=[5, 5, 0])
global_light = AmbientLight(color='#333333')

#normals_helper = VertexNormalsHelper(js_mesh, size=0.5, color="#ffcc00")

scene = Scene(children=[js_mesh, camera, point_light, global_light])

In [13]:
renderer = Renderer(
    camera=camera, 
    background="white", 
    background_opacity=1,
    scene=scene,
    controls=[OrbitControls(controlling=camera)],
    width=984,
    height=984,
)

display(renderer)

Renderer(background='white', camera=PerspectiveCamera(fov=20.0, position=(300.0, 50.0, 1200.0), quaternion=(0.…